**Exercício 2**

Para a realização deste trabalho, primeiramente, foram instaladas as ferramentas necessárias e de seguida, foi utilizado os módulos *BiConn.py* e *Hash.py* encontrados no material da disciplina e foi utilizado como base o script disponibilizado na aula TP1(2019).

In [1]:
from BiConn import *
from Hash import *
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

Após feita a importação dos módulos necessários, um *inicialization vector* de 16 bits é gerado.

In [2]:
my_salt = os.urandom(16)

Como descrito na aula, primeiramente o processo **Emitter** gera a chave a partir da *password* inserida, cria uma mensagem, cifra a mensagem com a chave e envia o criptograma pelo canal. Porém, para que haja uma autenticação do criptograma e dos metadados, a cifra foi alterada para o AES no modo GCM utilizando o módulo *aead* da biblioteca *cryptography*. Além disto, é feita a autenticação da chave utilizando o MAC disponível na função *mac* do módulo *Hash.py*.

In [3]:
def Emitter(conn):
    passwd = bytes(getpass('Emmiter password: '), 'utf-8')

    
    plaintext = os.urandom(1024)
    associated_data = os.urandom(256)
    iv = my_salt
    
    try:
        key = kdf(iv).derive(passwd)
        
        tag = mac(key, plaintext)
        
        aesgcm = AESGCM(key)
        ciphertext = aesgcm.encrypt(my_salt, plaintext, associated_data)
        
        # Cria um objecto contento do ciphertext, a tag e o associated_data
        obj = {'mess': ciphertext, 'tag': tag, 'aad': associated_data}
        conn.send(obj)  # envia o objecto pela conexão
    except:
        print("Erro no emissor")

    conn.close()

Ao receber o objecto, o **Receiver** verifica a *password* e após recuperar a informação, o criptograma 
é decifrado e a tag é verificada.

In [1]:
def Receiver(conn):
    passwd = bytes(getpass('Receiver password: '), 'utf-8')

    # verify passwd
    try:
        # Recuperar a informação
        obj = conn.recv()
        ciphertext = obj['mess']
        tag = obj['tag']
        associated_data = obj['aad']
        iv = my_salt
        # Gerar a chave e decifrar o ciphertext obtendo o plaintext
        key = kdf(iv).derive(passwd)
        
        aesgcm = AESGCM(key)
        plaintext = aesgcm.decrypt(my_salt, ciphertext, associated_data)
        # Verificação
        if mac(key, plaintext, tag) is None:
            print('OK')
        else:
            raise

    except:
        print('FAIL')

    conn.close()

In [5]:
BiConn(Emitter, Receiver, timeout=20).manual()

Emmiter password: ········
Receiver password: ········
OK
